# Домашнее задание к лекции "Базовые понятия статистики"

In [1]:
import pandas as pd
import numpy as np 

## Обязательная часть

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики. 

### Задание 1. Базовое изучение

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names) и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.



In [2]:
COLS_LIST = ['surgery' , 'Age' , 'Hospital Number' , 'rectal temperature', 'pulse', 'respiratory_rate' , 'temperature_of_extremities', 'peripheral pulse', 'mucous membranes', 'capillary refill time', 'pain', 'peristalsis' , 'abdominal distension', 'nasogastric tube', 'nasogastric reflux', 'nasogastric reflux PH', 'rectal examination - feces', 'abdomen' , 'packed cell volume', 'total_protein', 'abdominocentesis appearance', 'abdomcentesis total protein' , 'outcome', 'surgical lesion' , 'site of lesion', 'type, subtype', 'specific code', 'cp_data',]

In [3]:
horse_data = pd.read_csv('horse_data.csv',  names=COLS_LIST, sep=',', dtype=str)
horse_data[:5]

,surgery,Age,Hospital Number,rectal temperature,pulse,respiratory_rate,temperature_of_extremities,peripheral pulse,mucous membranes,capillary refill time,...,packed cell volume,total_protein,abdominocentesis appearance,abdomcentesis total protein,outcome,surgical lesion,site of lesion,"type, subtype",specific code,cp_data
0,2,1,530101,38.50,66,28,3,3,?,2,...,45.00,8.40,?,?,2,2,11300,00000,00000,2
1,1,1,534817,39.2,88,20,?,?,4,1,...,50,85,2,2,3,2,02208,00000,00000,2
2,2,1,530334,38.30,40,24,1,1,3,1,...,33.00,6.70,?,?,1,2,00000,00000,00000,1
3,1,9,5290409,39.10,164,84,4,1,6,2,...,48.00,7.20,3,5.30,2,1,02208,00000,00000,1
4,2,1,530255,37.30,104,35,?,?,6,2,...,74.00,7.40,?,?,2,2,04300,00000,00000,2


In [4]:
#list(horse_data.columns) 

### Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

In [5]:
horse_data = horse_data.replace("?", np.NaN)
horse_data['temperature_of_extremities'] = horse_data['temperature_of_extremities'].astype(float)

horse_data['temperature_of_extremities'][:10]

0    3.0
1    NaN
2    1.0
3    4.0
4    NaN
5    2.0
6    1.0
7    3.0
8    3.0
9    1.0
Name: temperature_of_extremities, dtype: float64

In [6]:
horse_data.temperature_of_extremities.dropna().astype(int).describe()

count    244.000000
mean       2.348361
std        1.045054
min        1.000000
25%        1.000000
50%        3.000000
75%        3.000000
max        4.000000
Name: temperature_of_extremities, dtype: float64

Выбросов нет. Температура конечностей преимущественно холодная в ~75% случаев. Пропусков в столбцах мало.

In [7]:
horse_data['Age'].isna().sum()

0

In [8]:
horse_data['Age'].astype(float).describe()

count    300.000000
mean       1.640000
std        2.173972
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        9.000000
Name: Age, dtype: float64

Выбросы есть - 9. В основном на лечение попадают лошади старше 6 мес (1 = Adult horse).

### Задание 3. Работа с пропусками

Рассчитать количество выбросов для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

In [9]:
horse_data_fin = horse_data.replace("?", np.NaN)

horse_data_fin['pulse'] = horse_data_fin['pulse'].astype(float)
horse_data_fin['Age'] = horse_data_fin['Age'].astype(float)

horse_data_fin.pulse.fillna(horse_data_fin.groupby('Age').pulse.transform('mean'), inplace=True)

horse_data_fin.groupby('Age').pulse.describe()

,count,mean,std,min,25%,50%,75%,max
Age,,,,,,,,
1.0,276.0,67.456693,22.956641,30.0,48.0,64.000000,80.5,140.0
9.0,24.0,123.363636,27.355224,84.0,100.0,123.681818,141.5,184.0


По всей видимости 9 - это все же молодые лошади, т.к. в источнике значения 2 не представлено.
Таким образом у молодых лошадей средний пульс в 2 раза ниже, чем у более старых.